# Low-Rank Autoregressive Tensor Completion (LATC)

- **CPU implementation**

In [ ]:
import numpy as np

def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def mat2ten(mat, tensor_size, mode):
    index = list()
    index.append(mode)
    for i in range(tensor_size.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, tensor_size[index].tolist(), order = 'F'), 0, mode)

def svt_tnn(mat, tau, theta):
    [m, n] = mat.shape
    if 2 * m < n:
        u, s, v = np.linalg.svd(mat @ mat.T, full_matrices = 0)
        s = np.sqrt(s)
        idx = np.sum(s > tau)
        mid = np.zeros(idx)
        mid[: theta] = 1
        mid[theta : idx] = (s[theta : idx] - tau) / s[theta : idx]
        return (u[:, : idx] @ np.diag(mid)) @ (u[:, : idx].T @ mat)
    elif m > 2 * n:
        return svt_tnn(mat.T, tau, theta).T
    u, s, v = np.linalg.svd(mat, full_matrices = 0)
    idx = np.sum(s > tau)
    vec = s[: idx].copy()
    vec[theta : idx] = s[theta : idx] - tau
    return u[:, : idx] @ np.diag(vec) @ v[: idx, :]

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def print_result(it, tol, var, var_hat):
    print('Iter: {}'.format(it))
    print('Tolerance: {:.6}'.format(tol))
    print('Imputation MAPE: {:.6}'.format(compute_mape(var, var_hat)))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(var, var_hat)))
    print()

from scipy.sparse import coo_matrix

def generate_Psi(dim_time, time_lags):
    Psis = []
    max_lag = np.max(time_lags)
    for i in range(len(time_lags) + 1):
        row = np.arange(0, dim_time - max_lag)
        if i == 0:
            col = np.arange(0, dim_time - max_lag) + max_lag
        else:
            col = np.arange(0, dim_time - max_lag) + max_lag - time_lags[i - 1]
        data = np.ones(dim_time - max_lag)
        Psi = coo_matrix((data, (row, col)),
                         shape = (dim_time - max_lag, dim_time)).tocsr()
        Psis.append(Psi)
    return Psis

def update_cg(var, r, d, Ad, rold):
    alpha = rold / np.inner(d, Ad)
    var = var + alpha * d
    r = r - alpha * Ad
    rnew = np.inner(r, r)
    d = r + (rnew / rold) * d
    return var, r, d, rnew

def ell_z(Z, double_A, Psi_T_Psi, lmbda, N, d):
    temp = lmbda * Z
    i = 0
    for k in range(d + 1):
        for h in range(d + 1):
            temp += (double_A[i].reshape([N, 1]) * (Z @ Psi_T_Psi[i]))
            i += 1
    return temp

def conj_grad_z(eq_right, Z, double_A, Psi_T_Psi, time_lags, lmbda, maxiter = 5):
    N, T = Z.shape
    z = np.reshape(Z, -1, order = 'F')
    r = np.reshape(eq_right - ell_z(Z, double_A, Psi_T_Psi,
                                    lmbda, N, len(time_lags)), -1, order = 'F')
    d = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        D = np.reshape(d, (N, T), order = 'F')
        Ad = np.reshape(ell_z(D, double_A, Psi_T_Psi,
                              lmbda, N, len(time_lags)), -1, order = 'F')
        z, r, d, rold = update_cg(z, r, d, Ad, rold)
    return np.reshape(z, (N, T), order = 'F')

def LATC(dense_tensor, sparse_tensor, time_lags, alpha, gamma, lmbda, theta,
         epsilon = 1e-4, maxiter = 100, K = 3):
    """Low-Rank Autoregressive Tensor Completion (LATC)"""

    dim = np.array(sparse_tensor.shape)
    dim_time = int(np.prod(dim) / dim[0])
    d = len(time_lags)
    sparse_mat = ten2mat(sparse_tensor, 0)
    pos_missing = np.where(sparse_mat == 0)
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    dense_test = dense_tensor[pos_test]
    del dense_tensor

    W_tensor = np.zeros(dim)
    Z_tensor = sparse_tensor.copy()
    Z = sparse_mat.copy()
    A = 0.001 * np.random.rand(dim[0], d)
    Psis = generate_Psi(dim_time, time_lags)
    it = 0
    ind = np.zeros((d, dim_time - time_lags[-1]), dtype = np.int_)
    for i in range(d):
        ind[i, :] = np.arange(time_lags[-1] - time_lags[i], dim_time - time_lags[i])
    Psi_T_Psi = []
    for k in range(d + 1):
        for h in range(d + 1):
            Psi_T_Psi.append(Psis[h].T @ Psis[k])
    last_mat = sparse_mat.copy()
    snorm = np.linalg.norm(sparse_mat, 'fro')
    while True:
        A_new = np.append(-np.ones(dim[0]).reshape([dim[0], 1]), A, axis = 1)
        double_A = []
        for k in range(d + 1):
            for h in range(d + 1):
                double_A.append(gamma * A_new[:, k] * A_new[:, h])
        for k in range(K):
            lmbda = min(lmbda * 1.05, 1e5)
            tensor_hat = np.zeros(dim)
            for s in range(len(dim)):
                tensor_hat += alpha[s] * mat2ten(svt_tnn(ten2mat(Z_tensor - W_tensor / lmbda, s),
                                                         alpha[s] / lmbda, theta), dim, s)
            eq_right = lmbda * ten2mat(tensor_hat + W_tensor / lmbda, 0)
            Z[pos_missing] = conj_grad_z(eq_right, Z, double_A, Psi_T_Psi,
                                         time_lags, lmbda)[pos_missing]
            Z_tensor = mat2ten(Z, dim, 0)
            W_tensor = W_tensor + lmbda * (tensor_hat - Z_tensor)
        for m in range(dim[0]):
            A[m, :] = np.linalg.lstsq(Z[m, ind].T, Z[m, time_lags[-1] :], rcond = None)[0]
        mat_hat = ten2mat(tensor_hat, 0)
        tol = np.linalg.norm((mat_hat - last_mat), 'fro') / snorm
        last_mat = mat_hat.copy()
        it += 1
        if it % 200 == 0:
            print_result(it, tol, dense_test, tensor_hat[pos_test])
        if (tol < epsilon) or (it >= maxiter):
            break
    print_result(it, tol, dense_test, tensor_hat[pos_test])

    return tensor_hat

## On the Seattle Freeway Traffic Speed Dataset

### Random Missing Data

In [ ]:
import numpy as np
import time

for r in [0.3, 0.7, 0.9]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    ## Random missing (RM)
    dense_tensor = np.load('tensor.npz')['arr_0'].transpose(0, 2, 1)
    dim1, dim2, dim3 = dense_tensor.shape
    np.random.seed(1000)
    sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

    for c in [1/10, 1/5, 1, 5, 10]:
        for theta in [5, 10, 15, 20, 25]:
            print('c = {}'.format(c))
            print('theta = {}'.format(theta))
            start = time.time()
            time_lags = np.arange(1, 7)
            alpha = np.ones(3) / 3
            lmbda = 1e-5
            gamma = c * lmbda
            tensor_hat = LATC(dense_tensor, sparse_tensor, time_lags,
                              alpha, gamma, lmbda, theta)
            end = time.time()
            print('Running time: %d seconds'%(end - start))
            print()

### Non-Random Missing Data

In [ ]:
import numpy as np
import time

for r in [0.3, 0.7]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    ## Non-random Missing (NM)
    dense_tensor = np.load('tensor.npz')['arr_0'].transpose(0, 2, 1)
    dim1, dim2, dim3 = dense_tensor.shape
    np.random.seed(1000)
    sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim3) + 0.5 - missing_rate)[:, None, :]

    for c in [1/10, 1/5, 1, 5, 10]:
        for theta in [5, 10, 15, 20, 25]:
            print('c = {}'.format(c))
            print('theta = {}'.format(theta))
            start = time.time()
            time_lags = np.arange(1, 7)
            alpha = np.ones(3) / 3
            lmbda = 1e-5
            gamma = c * lmbda
            tensor_hat = LATC(dense_tensor, sparse_tensor, time_lags,
                              alpha, gamma, lmbda, theta)
            end = time.time()
            print('Running time: %d seconds'%(end - start))
            print()

### Block-Out Missing

In [ ]:
import numpy as np
import time

for r in [0.3]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    ## Block-out Missing (BM)
    dense_tensor = np.load('tensor.npz')['arr_0'].transpose(0, 2, 1)
    dim1, dim2, dim3 = dense_tensor.shape
    block_window = 12
    np.random.seed(1000)
    vec = np.random.rand(int(dim2 * dim3 / block_window))
    temp = np.array([vec] * block_window)
    vec = temp.reshape([dim2 * dim3], order = 'F')
    sparse_tensor = dense_tensor * mat2ten(np.ones((dim1, dim2 * dim3)) * np.round(vec + 0.5 - missing_rate)[None, :], np.array([dim1, dim2, dim3]), 0)

    for c in [1/10, 1/5, 1, 5, 10]:
        for theta in [5, 10, 15, 20, 25]:
            print('c = {}'.format(c))
            print('theta = {}'.format(theta))
            start = time.time()
            time_lags = np.arange(1, 5)
            alpha = np.ones(3) / 3
            lmbda = 1e-5
            gamma = c * lmbda
            tensor_hat = LATC(dense_tensor, sparse_tensor, time_lags,
                              alpha, gamma, lmbda, theta)
            end = time.time()
            print('Running time: %d seconds'%(end - start))
            print()

## On the Portland Traffic Volume Dataset

### Random Missing

In [ ]:
import numpy as np
import time

for r in [0.3, 0.7, 0.9]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    # Random Missing (RM)
    dense_mat = np.load('volume.npy')
    dim1, dim2 = dense_mat.shape
    dim = np.array([dim1, 96, 31])
    dense_tensor = mat2ten(dense_mat, dim, 0)
    np.random.seed(1000)
    sparse_tensor = mat2ten(dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate), dim, 0)

    for c in [1/10, 1/5, 1, 5, 10]:
        for theta in [5, 10, 15, 20, 25]:
            print('c = {}'.format(c))
            print('theta = {}'.format(theta))
            start = time.time()
            time_lags = np.arange(1, 5)
            alpha = np.ones(3) / 3
            lmbda = 1e-5
            gamma = c * lmbda
            tensor_hat = LATC(dense_tensor, sparse_tensor, time_lags,
                              alpha, gamma, lmbda, theta)
            end = time.time()
            print('Running time: %d seconds'%(end - start))
            print()

### Non-Random Missing

In [ ]:
import numpy as np
import time

for r in [0.3, 0.7]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    # Non-random Missing (NM)
    dense_mat = np.load('volume.npy')
    dim1, dim2 = dense_mat.shape
    dim = np.array([dim1, 96, 31])
    dense_tensor = mat2ten(dense_mat, dim, 0)
    np.random.seed(1000)
    sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim[2]) + 0.5 - missing_rate)[:, None, :]

    for c in [1/10, 1/5, 1, 5, 10]:
        for theta in [5, 10, 15, 20, 25]:
            print('c = {}'.format(c))
            print('theta = {}'.format(theta))
            start = time.time()
            time_lags = np.arange(1, 5)
            alpha = np.ones(3) / 3
            lmbda = 1e-5
            gamma = c * lmbda
            tensor_hat = LATC(dense_tensor, sparse_tensor, time_lags,
                              alpha, gamma, lmbda, theta)
            end = time.time()
            print('Running time: %d seconds'%(end - start))
            print()

### Block-Out Missing

In [ ]:
import numpy as np
import time

for r in [0.3]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    ## Block-out Missing (BM)
    dense_mat = np.load('volume.npy')
    dim1, dim2 = dense_mat.shape
    dim = np.array([dim1, 96, 31])
    dense_tensor = mat2ten(dense_mat, dim, 0)
    block_window = 4
    np.random.seed(1000)
    vec = np.random.rand(int(dim2 / block_window))
    temp = np.array([vec] * block_window)
    vec = temp.reshape([dim2], order = 'F')
    sparse_tensor = mat2ten(dense_mat * np.round(vec + 0.5 - missing_rate)[None, :], dim, 0)

    for c in [1/10, 1/5, 1, 5, 10]:
        for theta in [5, 10, 15, 20, 25]:
            print('c = {}'.format(c))
            print('theta = {}'.format(theta))
            start = time.time()
            time_lags = np.arange(1, 5)
            alpha = np.ones(3) / 3
            lmbda = 1e-5
            gamma = c * lmbda
            tensor_hat = LATC(dense_tensor, sparse_tensor, time_lags,
                              alpha, gamma, lmbda, theta)
            end = time.time()
            print('Running time: %d seconds'%(end - start))
            print()

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>